# Exploratory Analysis of Import Data

This notebook explores the PIERS Bill of Lading data, obtained from S&P's Global Trade Analytics Suite. See the README.md file for more info on the overall project, data pre-processing, and column definitions. 

In [1]:
#import libraries
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns


#display settings
pd.set_option('display.max_columns', None)
%matplotlib inline

#enable string cache for polars categoricals
pl.enable_string_cache()

## Basic Summary Stats

In [3]:
#set paths
imports_path = 'data/clean/imports/'
exports_path = 'data/clean/exports/'

#get schema and col names
imports_schema = pl.read_parquet_schema(source= imports_path+'piers_imports_2005.parquet')
imports_colnames = imports_schema.keys()
exports_schema = pl.read_parquet_schema(source=exports_path+'piers_exports_complete.parquet')
exports_colnames = exports_schema.keys()

#init lazy dataframes
imports_lzdf = pl.scan_parquet(imports_path+'*.parquet', parallel='columns')
exports_lzdf = pl.scan_parquet(exports_path+'piers_exports_complete.parquet', parallel='columns')

#get number of observations
imports_n = imports_lzdf.select(pl.count()).collect().item()
exports_n = exports_lzdf.select(pl.count()).collect().item()
print('The imports dataset has {:,} rows and {} columns.'.format(imports_n, len(imports_schema)))
print('The exports dataset has {:,} rows and {} columns.'.format(exports_n, len(exports_schema)))

The imports dataset has 178,875,896 rows and 44 columns.
The exports dataset has 68,769,489 rows and 30 columns.


In [4]:
#view head of imports dataframe 
imports_lzdf.limit(n=3).collect()

weight,weight_unit,qty,qty_type,teus,value_est,date_arrival,container_piece_count,commod_short_desc_qty,origin_territory,origin_region,arrival_port_code,arrival_port_name,departure_port_code,departure_port_name,dest_final,coast_region,clearing_district,place_receipt,shipper_name,shipper_address,consignee_name,consignee_address,notify_party1_name,notify_party1_address,notify_party2_name,notify_party2_address,commod_desc_raw,container_id_marks,marks_desc,hs_code,joc_code,commod_short_desc,container_ids,carrier_name,carrier_scac,vessel_name,voyage_number,precarrier,vessel_id,inbond_code,transport_mode,bol_number,bol_id
f64,cat,f64,cat,f64,f64,datetime[μs],i32,str,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,str,str,str,str,str,str,str,str,str,str,str,cat,cat,str,str,cat,cat,str,str,f64,i32,f64,cat,str,str
0.0,"""KG""",1510.0,"""CTN""",0.0,0.0,2005-11-14 00:00:00,1,"""1510""","""CHINA (MAINLAN…","""NORTH EAST ASI…","""4601""","""NEW YORK""","""58201""","""HONG KONG""",""" ""","""EAST""",null,"""CHINA P""","""SUNNY INDUSTRY…","""33/FL SOUTHORN…","""SHAW CREATIONS…","""65 CLYDE RD ST…","""MAERSK CUSTOMS…","""230-79 INTERNA…",null,null,null,"""""","""LC""","""660199""","""""","""""","""MSKU6622877""","""MAERSK LINE""","""MLSL""","""MAGLEBY MAERSK…","""512""",null,8715871,null,null,"""850353063""","""MLSL_850353063…"
0.0,"""KG""",29.0,"""CTN""",0.0,0.0,2005-08-29 00:00:00,1,"""29""","""JAPAN""","""NORTH EAST ASI…","""2709""","""LONG BEACH""","""58886""","""TOKYO""",""" ""","""WEST""","""LOS ANGELES, C…","""TOKYO""","""NIPPON SHUPPAN…","""4-4-6, SAMBANC…","""CASA ONO COMER…","""RUA FERNAO DIA…","""OZ ARCHITECTS""","""6621 N SCOTTSD…",null,null,null,"""""","""LC""","""4902""","""""","""""","""KLFU5956177""","""K LINE""","""KLIN""","""AKASHI BRIDGE""","""18""",null,9043782,null,null,"""SFLSA001640""","""KLIN_SFLSA0016…"
0.0,"""KG""",7.0,"""CTN""",0.0,0.0,2005-08-01 00:00:00,1,"""7""","""CHINA (MAINLAN…","""NORTH EAST ASI…","""3002""","""TACOMA""","""57035""","""SHANGHAI""",""" ""","""WEST""","""SEATTLE, WASHI…","""SHANGHAI""","""WELLTON INTERN…","""""","""WELLTON EXPRES…","""750 STUART GRA…","""WELLTON EXPRES…","""750 STUART GRA…",null,null,null,"""""","""LC""","""630232""","""""","""""","""MOTU0119951""","""MOL""","""OSKL""","""HYUNDAI COMMOD…","""135""",null,9035981,null,null,"""483354180""","""OSKL_483354180…"


In [5]:
#view head of exports dataframe 
exports_lzdf.limit(n=3).collect()

shipper,shipper_address,weight,weight_unit,quantity,quantity_type,teus,carrier_name,carrier_scac,vessel_name,voyage_number,bol_id,imo_num,value_est,departure_port_code,departure_port_name,container_ids,container_piece_count,coast_region,commod_desc_raw,commod_short_desc,hs_code,joc_code,commod_short_desc_qty,date_departure,origin,dest_territory,dest_region,dest_port_code_declared,dest_port_name
str,str,f64,cat,f64,cat,f64,cat,cat,str,str,str,i32,f64,cat,cat,str,i32,cat,str,str,str,str,str,datetime[μs],str,cat,cat,i32,cat
null,null,0.0,"""KG""",28.0,"""UNT""",0.0,"""HOEGH UGLAND A…","""AUTO""","""HUAL SEOUL""","""10""","""020""",9285495,0.0,"""1103""","""WILMINGTON DE""","""""",0,"""EAST""",null,"""""","""870390""","""""","""1; 1; 1; 3; 3;…",2005-12-05 00:00:00,""" ""","""SAUDI ARABIA""","""MIDDLE EAST""",51721,"""JEDDAH"""
null,null,0.0,"""KG""",11.0,"""UNT""",0.0,"""HOEGH UGLAND A…","""AUTO""","""OCEAN SPIRIT""","""93""","""03001""",8321747,0.0,"""1803""","""JACKSONVILLE""","""""",0,"""EAST""",null,"""""","""870390""","""""","""1; 1; 1; 1; 1;…",2005-12-26 00:00:00,""" ""","""SAUDI ARABIA""","""MIDDLE EAST""",51721,"""JEDDAH"""
null,null,0.0,"""KG""",2.0,"""LOT""",0.0,"""P&O NEDLLOYD""","""PONL""","""MAERSK NANHAI""","""5344""","""LNAM54066460""",9306483,0.0,"""5301""","""HOUSTON""","""GLDU4096505 GL…",2,"""GULF""",null,"""""","""285100""","""""","""1; 1""",2005-12-10 00:00:00,""" ""","""EGYPT""","""MEDITERRANEAN""",72901,"""ALEXANDRIA"""


In [6]:
#init df and get stats labels column
imports_summarystats_df = imports_lzdf.select(pl.first()).collect().describe().select(pl.first()).to_pandas()
#loop through columns and get descriptive stats
for column in imports_colnames:
    imports_summarystats_df[column] = imports_lzdf.select(pl.col(column)).collect().describe().select(column).to_pandas()
#display
imports_summarystats_df

,describe,weight,weight_unit,qty,qty_type,teus,value_est,date_arrival,container_piece_count,commod_short_desc_qty,origin_territory,origin_region,arrival_port_code,arrival_port_name,departure_port_code,departure_port_name,dest_final,coast_region,clearing_district,place_receipt,shipper_name,shipper_address,consignee_name,consignee_address,notify_party1_name,notify_party1_address,notify_party2_name,notify_party2_address,commod_desc_raw,container_id_marks,marks_desc,hs_code,joc_code,commod_short_desc,container_ids,carrier_name,carrier_scac,vessel_name,voyage_number,precarrier,vessel_id,inbond_code,transport_mode,bol_number,bol_id
0,count,1.788759e+08,178875896,1.788759e+08,150054988,1.788759e+08,1.788759e+08,178875896,1.788759e+08,178875850,178004511,178004511,178842823,178842823,178399200,178399200,178875896,178767698,138267806,178028936,177152819,149352161,177316770,158522570,131500774,147349739,11190643,80229284,153344894,178875896,178875896,178875850,178875850,178875850,178875896,178743544,178875872,167484536,178875896,0.0,1.673943e+08,4.599980e+07,54698436,178875896,178875872
1,null_count,0.000000e+00,0,0.000000e+00,28820908,0.000000e+00,0.000000e+00,0,0.000000e+00,46,871385,871385,33073,33073,476696,476696,0,108198,40608090,846960,1723077,29523735,1559126,20353326,47375122,31526157,167685253,98646612,25531002,0,0,46,46,46,0,132352,24,11391360,0,178875896.0,1.148159e+07,1.328761e+08,124177460,0,24
2,mean,4.740686e+04,None,1.203557e+03,None,1.285770e+00,7.808732e+04,None,1.440886e+00,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,9.338100e+06,3.066917e+02,None,None,None
3,std,1.721627e+06,None,4.090204e+05,None,3.112699e+00,2.853759e+06,None,3.409732e+00,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,2.850288e+05,1.059749e+03,None,None,None
4,min,0.000000e+00,None,-7.456109e+07,None,0.000000e+00,0.000000e+00,2005-01-01 00:00:00,0.000000e+00,,None,None,None,None,None,None,None,None,None,None,,,,,,,& OPTCO,,,,,None,None,,,None,None,,,NaN,0.000000e+00,0.000000e+00,None,$70050200001,-1_CSHSE0009996
5,25%,0.000000e+00,None,1.000000e+00,None,0.000000e+00,0.000000e+00,None,1.000000e+00,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,9.248112e+06,0.000000e+00,None,None,None
6,50%,1.466000e+03,None,3.800000e+01,None,2.200000e-01,6.571000e+03,None,1.000000e+00,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,9.332884e+06,0.000000e+00,None,None,None
7,75%,1.501000e+04,None,4.000000e+02,None,2.000000e+00,4.552600e+04,None,1.000000e+00,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,9.462706e+06,0.000000e+00,None,None,None
8,max,6.172936e+09,None,3.531460e+09,None,1.730750e+03,9.981402e+09,2023-11-30 00:00:00,9.990000e+02,9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9...,None,None,None,None,None,None,None,None,None,None,“COOPERATIVA OPERAI CAVATORI DEL BOTTICINO,ÞORRASALIR 13,C MCCRACKEN,ŠILUTES STR 9,C MCCRACKEN,ŠILUTES STR 9,ZYP COATINGS,co RMI Titanium Company LLC 208, WORKING GLOVES,}No MarksNo MarksNo Marks,; ; ; ; ; ; ; ; ; ; ; ,None,None,"ZIRCONIUM; SAND,FLOUR,SPONGE,POWDER VALVES; EN...",vTA1108,None,None,ZYGI,^144,NaN,9.975363e+06,7.122000e+03,None,_YLC158932,zimu_ZIMUZHJ0000952


In [7]:
#NOTE at the moment the exports dataset may fit in memory, in which case the below code could be accomplished more efficiently
#by executing the following line; however, the main code below should run even when the dataset does not fit in memory. 
#exports_lzdf.collect().describe()

#init df and get stats labels column
exports_summarystats_df = exports_lzdf.select(pl.first()).collect().describe().select(pl.first()).to_pandas()
#loop through columns and get descriptive stats
for column in exports_colnames:
    exports_summarystats_df[column] = exports_lzdf.select(pl.col(column)).collect().describe().select(column).to_pandas()
#display
exports_summarystats_df

,describe,shipper,shipper_address,weight,weight_unit,quantity,quantity_type,teus,carrier_name,carrier_scac,vessel_name,voyage_number,bol_id,imo_num,value_est,departure_port_code,departure_port_name,container_ids,container_piece_count,coast_region,commod_desc_raw,commod_short_desc,hs_code,joc_code,commod_short_desc_qty,date_departure,origin,dest_territory,dest_region,dest_port_code_declared,dest_port_name
0,count,52901485,51534703,6.876949e+07,68769489,6.876949e+07,61810793,6.876949e+07,68595453,68768115,63384893,67625608,68769456,6.323115e+07,6.876949e+07,68720145,68720145,68769489,6.876949e+07,68769013,53168182,68769437,68769437,68769437,68769437,68769489,68769489,68279599,68279599,6.829928e+07,68299282
1,null_count,15868004,17234786,0.000000e+00,0,0.000000e+00,6958696,0.000000e+00,174036,1374,5384596,1143881,33,5.538337e+06,0.000000e+00,49344,49344,0,0.000000e+00,476,15601307,52,52,52,52,0,0,489890,489890,4.702070e+05,470207
2,mean,None,None,1.152067e+05,None,3.965311e+04,None,1.672979e+00,None,None,None,None,None,9.159625e+06,1.415664e+05,None,None,None,1.923483e+00,None,None,None,None,None,None,None,None,None,None,4.907175e+04,None
3,std,None,None,2.375519e+06,None,3.471335e+06,None,5.518306e+00,None,None,None,None,None,5.438769e+05,6.466967e+06,None,None,None,1.109810e+01,None,None,None,None,None,None,None,None,None,None,1.906010e+04,None
4,min,,,0.000000e+00,None,-1.467123e+09,None,0.000000e+00,None,None,,,,1.960000e+02,0.000000e+00,None,None,,0.000000e+00,None,,,,,,2005-01-01 00:00:00,,None,None,0.000000e+00,None
5,25%,None,None,0.000000e+00,None,0.000000e+00,None,0.000000e+00,None,None,None,None,None,9.143568e+06,0.000000e+00,None,None,None,1.000000e+00,None,None,None,None,None,None,None,None,None,None,3.370000e+04,None
6,50%,None,None,1.360800e+02,None,1.300000e+01,None,0.000000e+00,None,None,None,None,None,9.295220e+06,4.080000e+02,None,None,None,1.000000e+00,None,None,None,None,None,None,None,None,None,None,5.205100e+04,None
7,75%,None,None,1.919696e+04,None,1.780000e+02,None,2.000000e+00,None,None,None,None,None,9.398424e+06,4.627100e+04,None,None,None,1.000000e+00,None,None,None,None,None,None,None,None,None,None,5.820100e+04,None
8,max,SUNSET TRANSPORTATION SERVICE,,9.529981e+08,None,3.716633e+09,None,3.729250e+03,None,None,xin yang shan,|SAL5,PE10518042,9.980629e+06,1.000000e+10,None,None,~POPLAR AVE |SAVALY762976 _38197 WFHU5153172 W...,4.136000e+03,None,ÙTEEL GUARD RAIL; ÙTEEL GUARD RAILS,"ZIRCONIUM; SAND,FLOUR,SPONGE,POWDER ZIRCONIUM ...",ddedo,9669000 7746000 1301000 6830000,9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9...,2023-11-30 00:00:00,,None,None,9.993000e+04,None


## Duplicate BOLs 

In [15]:
#count unique bol_scac IDs
import_bols_unique_n = imports_lzdf.select(pl.col('bol_id')).unique().select(pl.count()).collect().item()
export_bols_unique_n = exports_lzdf.select(pl.col('bol_id')).unique().select(pl.count()).collect().item()

print('{:,} out of {:,} rows ({:.2f}%) in the imports dataset contain duplicated BoLs.'.format(imports_n-import_bols_unique_n, imports_n, (imports_n-import_bols_unique_n)/imports_n*100))
print('{:,} out of {:,} rows ({:.2f}%) in the exports dataset contain duplicated BoLs.'.format(exports_n-export_bols_unique_n, exports_n, (exports_n-export_bols_unique_n)/exports_n*100))

1,158,548 out of 178,875,896 rows (0.65%) in the imports dataset contain duplicated BoLs.
3,862,317 out of 68,769,489 rows (5.62%) in the exports dataset contain duplicated BoLs.


Possible reasons:
- bol includes items on more than one ship
- data entry errors
- aggregations of some kind by S&P
- ???


## Value and Volumes by Year:

In [ ]:
#get year col
pldf['year'] = pldf.date_arrival.dt.to_period('Y')
#group value and volume by year
activityacrosstime_df = pldf[['year', 'teus', 'value_est']].groupby('year').sum()
#plot
sns.barplot(data=activityacrosstime_df, x='year', y='value_est');
plt.title('Total Value of Imports Over Time')
plt.xticks(rotation=45);

In [ ]:
#plot
sns.barplot(data=activityacrosstime_df, x='year', y='teus');
plt.title('Total Volume (TEUs) of Imports Over Time')
plt.xticks(rotation=45);

I guess value and volume records weren't kept before ~2015 ?!?

In [ ]:
#group value and volume by year
until2012_df = pldf[pldf['year']< pd.Period(2013)]
until2012_df = until2012_df[['year', 'teus', 'value_est']].groupby('year').sum()
#plot
sns.barplot(data=until2012_df, x='year', y='teus');
plt.title('Total Volume (TEUs) of Imports Over Time')
plt.xticks(rotation=45);

Must be missing data here?

In [ ]:
#get year col
pldf['year'] = pldf.date_arrival.dt.to_period('Y')
#group value and volume by year
activityacrosstime_df = pldf[['year', 'container_piece_count']].groupby('year').sum()
#plot
sns.barplot(data=activityacrosstime_df, x='year', y='container_piece_count');
plt.title('Total Volume (Container Piece Count) of Imports Over Time')
plt.xticks(rotation=45);

In [ ]:
pldf.head()

## Carriers Over Time

In [ ]:
carriersovertime_df = pldf[['year', 'carrier_scac']].groupby('year').nunique()

In [ ]:
sns.barplot(data=carriersovertime_df, x='year', y='carrier_scac');
plt.title('Number of Unique Carriers Over Time');
plt.xticks(rotation=45);

Was there really a spike in carriers in 2010 and 2012? Or does this indicate changes in the way SCAC codes are assigned?

### Market share of the 50 largest carriers by estimated value 

In [ ]:
#get largest carriers
carriers_df = pldf[pldf.year > pd.Period(2014)]
carriers_df = carriers_df[['year', 'carrier_scac', 'value_est']].groupby(['year', 'carrier_scac']).sum()

In [ ]:
carriers_df.columns = ['value_usd']
carriers_df.sort_values('value_usd', ascending=False, inplace=True)
carriers_df.sort_values('year', inplace=True)

In [ ]:
carriers_df.reset_index(inplace=True)
carriers_df.head()